In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline

In [ ]:
avo = 6.022e23
m2µm  = 1e6
def surf_a(d, l):
    return np.pi *  d  * l
def vol(d , l ):
    return np.pi*l*(d/2)**2
def cross_a(d):
    return np.pi *  (d/2)**2

base_cyt = 10432667.412756/((3.1e-16 )*avo)   #moles ATP /sec per m^3 
base_mem = 909175.81/((5.68e-10 )*avo)      #moles ATP /sec per m^2

DATP = 300  ## µm^2/s
def base_cons(volume, sarea , bc=base_cyt, bm = base_mem ):
    return bc*volume + bm * sarea

d_range = np.arange(0.1,1,0.1)  # diameters in µm
l_range  = np.arange(1,5,0.1)  # length from mito midpoint to midpoint b/w  mito

X = np.copy(d_range)
Y = np.copy(l_range)
X, Y = np.meshgrid(d_range, l_range)
Z = base_cons(vol(X/m2µm,Y/m2µm),surf_a(X/m2µm,Y/m2µm))



## Good first - add contours of single lines for fixed diameters

In [ ]:
ax = plt.figure().add_subplot(1, 1, 1, projection='3d')
surf = ax.plot_surface(X, Y, Z, linewidth=0, antialiased=True)
ax.set_xlabel('Diameter µm')
ax.set_ylabel('Intervari. length µm')
ax.set_zlabel('Rest Consumption (ATP moles x 10^-17 /s)')
ax.dist = 13
plt.show()

## Consumption estimates for activity:  https://www.jneurosci.org/content/42/6/954/tab-figures-data

Fig 3 (F) == Power ATP molecules per sec Ca2+
Fig 4 (F) == Power ATP molecules per sec Na+

comitted_ca and committed_na are ATP molecules per Action Potential
Power demand taken from supple tables, ATP molecules per sec

$M6Ib$ 

6.82e7    ATP molecules/s for Ca2+

6.49e7    for Na

5.68e-10       SA:m^2


$M6Is$

0.33e7

0.3e7

2.34e-10


$M13Ib$

9.97e7

12.1e7

5.43e-10


$M13Is$

0.31e7

0.32e7

2.51e-10

$M12Ib$

8.46e7

7.93e7

5.95e-10

$M12Is$

0.49e7

0.37e7

2.95e-10

In [ ]:
terms = {
    "M61b" : {'power_sa': (6.28e7+6.49e7)/avo ,'power_vol': 2.27e7 /avo ,'area':5.68e-10 , 'volume':3.1e-16 },\
    'M61s':  {'power_sa': (0.33e7+0.3e7)/avo  ,'power_vol': 0.29e7 /avo  ,'area':2.34e-10 , 'volume':9.01e-17 }, \
    'M131b': {'power_sa': (9.97e7+12.1e7)/avo ,'power_vol': 2.4e7  /avo ,'area':5.43e-10 , 'volume':3.92e-16 } 
            } 

In [ ]:
#X and Y are diameters and lengths respectively.  ZA and ZV are  areal and volumic consumptions respectively
#

In [ ]:
fig = plt.figure(figsize=plt.figaspect(0.45))
ax1 = fig.add_subplot(1,2,1, projection ='3d')

ZA = terms['M61b']['power_sa']*surf_a(X/m2µm,Y/m2µm)/terms['M61b']['area']/1
surf = ax1.plot_surface(X, Y, ZA, linewidth=0, antialiased=True)
ZA2 = terms['M61s']['power_sa']*surf_a(X/m2µm,Y/m2µm)/terms['M61s']['area']
surf = ax1.plot_surface(X, Y, ZA2, linewidth=0, antialiased=True)
ax1.set_xlabel('Diameter µm')
ax1.set_ylabel('Compartment length µm')
ax1.set_zlabel('Surf_area Consumption (ATP moles x 10^-17 /s)')

ax2= fig.add_subplot(1,2,2,projection='3d')
ZV = terms['M61b']['power_vol']*vol(X/m2µm,Y/m2µm)/terms['M61b']['volume']/1
surf = ax2.plot_surface(X, Y, ZV, linewidth=0, antialiased=True)
Z2V = terms['M61s']['power_vol']*vol(X/m2µm,Y/m2µm)/terms['M61s']['volume']
surf = ax2.plot_surface(X, Y, Z2V, linewidth=0, antialiased=True)
ax2.set_xlabel('Diameter µm')
ax2.set_ylabel('Compartment length µm')
ax2.set_zlabel('Volume Consumption (ATP moles x 10^-18 /s)')
ax2.dist = 13
plt.show()

In [ ]:
#  Quantifying largest consumptions in terms  of mM/s, presuming 
#the  largest consumptions come from the thinnest and longest axons

print((max(ZA[-1])/vol(     min(d_range)/m2µm  ,  max(l_range)/m2µm   ), \
max(ZV[-1])/vol(     min(d_range)/m2µm  ,  max(l_range)/m2µm   ) ),  str('mM/s'))

### ATP Drops for endogenous EFR of M6Ib

In [ ]:
ax1 = plt.figure().add_subplot(1, 1, 1, projection='3d')
ATP_drop = (ZA+ZV)*(Y/m2µm)/(2*np.pi*(DATP/(m2µm)**2 )*(X/m2µm)**2/4)
ATP_drop_base = (base_cons(vol(X/m2µm,Y/m2µm), surf_a(X/m2µm,Y/m2µm)))*(Y/m2µm)/(2*np.pi*(DATP/(m2µm)**2 )*(X/m2µm)**2/4)
surf1 = ax1.plot_surface(X, Y,ATP_drop*100/18, linewidth=0, antialiased=True)
ax1.set_xlabel('Diameter µm')
ax1.set_ylabel('Compartment length µm')
ax1.set_zlabel('Difference in [ATP] from mito to neighbor mM/s')
ax1.dist = 13
plt.show()

ax2 = plt.figure().add_subplot(1, 1, 1, projection='3d')
surf2 = ax2.plot_surface(X, Y,ATP_drop*100/18+ATP_drop_base, linewidth=0, antialiased=True)
ax2.set_xlabel('Diameter µm')
ax2.set_ylabel('Compartment length µm')
ax2.set_zlabel('Difference in [ATP] from mito to neighbor mM/s')
ax2.dist = 13
plt.show()


ax3 = plt.figure().add_subplot(1, 1, 1, projection='3d')
surf3 = ax3.plot_surface(X, Y,ATP_drop_base, linewidth=0, antialiased=True)
ax3.set_xlabel('Diameter µm')
ax3.set_ylabel('Compartment length µm')
ax3.set_zlabel('Difference in [ATP] from mito to neighbor mM/s')
ax3.dist = 13
plt.show()

In [ ]:
(100/18)*ATP_drop[-1] , ATP_drop_base[-1]

### Apply to " squid " system with 2.18 (AdT), PhT 3.3+7.5, Kph  = 39.6, ADP_1???

In [ ]:
def beta(ADP_1,PhT=7.5+3.3,AdT=2.18,KPh=39.6):
    return KPh*AdT*PhT/((AdT + (KPh - 1)*ADP_1)**2)

In [ ]:
beta(.1)

In [ ]:
ax = plt.figure().add_subplot(1, 1, 1, projection='3d')

Ph_ATP_drop = (ZA+ZV)*(Y/m2µm)/(2*np.pi*((DATP+beta(0.02)*450)/(m2µm)**2 )*(X/m2µm)**2/4)
ATP_drop = (ZA+ZV)*(Y/m2µm)/(2*np.pi*((DATP)/(m2µm)**2 )*(X/m2µm)**2/4)

surf1 = ax.plot_surface(X, Y,Ph_ATP_drop*100/18, linewidth=0, antialiased=True)
surf2 = ax.plot_surface(X, Y,ATP_drop*100/18, linewidth=0, antialiased=True, alpha =0.4)

ax.set_xlabel('Diameter µm')
ax.set_ylabel('Compartment length µm')
ax.set_zlabel('Difference in [ATP] from mito to neighbor mM/s')
ax.dist = 13
plt.show()

In [ ]:
ax = plt.figure().add_subplot(1, 1, 1, projection='3d')

Ph02_ATP_drop = (ZA+ZV)*(Y/m2µm)/(2*np.pi*((DATP+beta(0.02)*450)/(m2µm)**2 )*(X/m2µm)**2/4)
Ph08_ATP_drop = (ZA+ZV)*(Y/m2µm)/(2*np.pi*((DATP+beta(0.08)*450)/(m2µm)**2 )*(X/m2µm)**2/4)

surf1 = ax.plot_surface(X, Y,Ph02_ATP_drop*100/18, linewidth=0, antialiased=True)
surf2 = ax.plot_surface(X, Y,Ph08_ATP_drop*100/18, linewidth=0, antialiased=True, alpha =0.4)

ax.set_xlabel('Diameter µm')
ax.set_ylabel('Compartment length µm')
ax.set_zlabel('Difference in [ATP] from mito to neighbor mM/s')
ax.dist = 13
plt.show()

## What treatments go together?

### Ca2+ and Na+ go with surface area
### base rates are already subdivided
### "signaling" a la KJ 2022 is volumic (though shepard would probably object for en passant varicosities. Try this version later.

An analytical solution for ATPdrop without phos is as follows:
$$ \Delta [ATP] = \frac{2 l^2}{D_{ATP} d} \left(C^v \frac{d}{4} + C^{SA} \right) $$

where $l,d,C^V,C^{SA} $ are interbourton length, axon diameter, volumic ATP consumption density, and surface areal ATP consumption density